# Data Preparation , Once and for all !

Let's summarize the data preparation we are performing for the different models we are building 

Idea : 
    Could be good to update our classe to use this without having to copy and paste 

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [ ]:
from transplant.tools.dataset import Dataset 

dataset = Dataset()

train_static_0, test_static_0 = dataset.get_static()

In [ ]:
train_static_str_to_num=train_static_0.apply(pd.to_numeric,errors='coerce').dropna(1, how="all")

mean_train_static = train_static_str_to_num.mean()

train_static_filled=train_static_str_to_num.fillna(mean_train_static)

In [ ]:
test_static_filled=test_static_0.apply(pd.to_numeric,errors='coerce').dropna(1, how="all").fillna(mean_train_static)

Here we make sure we got the same columns in train and test (static)

In [ ]:
drop_test=[]
drop_train=[]
train_static_columns=train_static_filled.columns
test_static_columns=test_static_filled.columns

for i in train_static_filled.columns :
    if not(i in test_static_columns) :
        drop_train+=[i]
        
for i in test_static_columns :
    if not(i in train_static_filled.columns) :
        drop_test+=[i]
drop_test , drop_train

train_static_filled=train_static_filled.drop(drop_train, axis=1)
test_static_filled=test_static_filled.drop(drop_test, axis=1)


In [ ]:
train_static_filled.shape, test_static_filled.shape

A function to rescale our data

In [ ]:
def center_reduce_data(X_train,X_test):
    mean_train=X_train.mean()
    std_train=X_test.std()

    return (X_train-mean_train)/std_train, (X_test-mean_train)/std_train
    

Train static we use for the moment

In [ ]:
train_static=train_static_filled
test_static=test_static_filled


# Dynamic

In [ ]:
train_dynamic_0, test_dynamic_0 = dataset.get_dynamic()

In [ ]:
train_dynamic_0.columns 

## ATTENTION ! HERE 30 patients missing in dynamic ...

In [ ]:
static_id=train_static['id_patient'].sort_values().unique()
dynamic_id=train_dynamic_0['id_patient'].unique()

In [ ]:
static_id.shape , dynamic_id.shape

In [ ]:
in_static_not_in_dynamic=[]
in_dynamic_not_in_static=[]
for i in static_id :
    if not(i in dynamic_id):
        in_static_not_in_dynamic+=[i]
for i in dynamic_id:
    if not(i in static_id):
        in_dynamic_not_in_static+=[i]
in_static_not_in_dynamic, in_dynamic_not_in_static

In [ ]:
def plot_serie_patient(X,patient_id,variable) :
    # On choisit la variable à observer dans table_col au dessus 
    df=X[X['id_patient']==patient_id].sort_values('time') ;
    df['time']-=df['time'].iloc[0] ; # Ici j'initialise le temps à 0. Càd le début est à 0.
    df=df.set_index('time') ;
    plt.title('Evolution de la variable "'+variable+'" du patient '+str(patient_id)+"| Durée de l'opération : "+str(df.index[-1]), y=1.05, size=25)
    plt.plot(df[variable])

In [ ]:
fig = plt.figure(figsize=(20,16))
plot_serie_patient(train_dynamic_0,1,'BIS SR')
plot_serie_patient(train_dynamic_0,2,'BIS SR')
plot_serie_patient(train_dynamic_0,3,'BIS SR')

### Replace NaN dynamic with mean train_set

In [ ]:
mean_dynamic_train=train_dynamic_0.groupby(['id_patient']).mean().mean()

In [ ]:
train_dynamic=train_dynamic_0.fillna(mean_dynamic_train)
test_dynamic=test_dynamic_0.fillna(mean_dynamic_train)

In [ ]:
train_dynamic_flat=train_dynamic.groupby(['id_patient'], as_index=False).mean()
test_dynamic_flat=test_dynamic.groupby(['id_patient'],as_index=False).mean()

In [ ]:
train_dynamic_flat.head()

### Sum up before merging

In [ ]:
train_static.columns

## Merge dynamic and static

In [ ]:
def merge_dyn_sta(X_train_static,X_train_dynamic,X_test_static,X_test_dynamic):
    return pd.merge(X_train_static, X_train_dynamic, on='id_patient') , pd.merge(X_test_static, X_test_dynamic, on='id_patient') 

In [ ]:
train_glob, test_glob = merge_dyn_sta(train_static,train_dynamic_flat,test_static,test_dynamic_flat)

In [ ]:
train_glob.columns

In [ ]:
train_glob

## Final Data 

In [ ]:
dic_to_One_Hot = {0 : [1,0], 1 : [0,1]}

In [ ]:
X_train=np.array(train_glob.drop(['target'], axis=1))
X_test=np.array(test_glob.drop(['target'], axis=1))

y_train_cls=np.array(train_glob['target'])
y_train_hot=np.array(list(train_glob['target'].map(dic_to_One_Hot)))

y_test_cls=np.array(test_glob['target'])
y_test_hot=np.array(list(test_glob['target'].map(dic_to_One_Hot)))


In [ ]:
X_train

## All this for that !


In [1]:
from transplant.tools.dataset import Dataset 

dataset = Dataset()

X_train, X_test , y_train, y_test = dataset.get_data_pierre()

C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\dataset.py:134: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  mean_dynamic_train=train_dynamic_0.groupby(['id_patient']).mean().mean()
C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\dataset.py:139: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  train_dynamic_flat=train_dynamic.groupby(['id_patient'], as_index=False).mean()
C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\dataset.py:140: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  test_dynamic_flat=test_dynamic.groupby(['id_patient'],as_index=False).mean()
